##### Copyright 2025 Google LLC.

In [1]:
import os
from getpass import getpass

GOOGLE_API_KEY = getpass("Enter your GOOGLE API KEY: ")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print(" Google API key set successfully!")


 Google API key set successfully!


#### 1.3: Import ADK components



In [2]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print(" ADK components imported successfully.")

 ADK components imported successfully.


### 1.4: Helper functions

Helper function that manages a complete conversation session, handling session
creation/retrieval, query processing, and response streaming. It supports
both single queries and multiple queries in sequence.

Example:

```
>>> await run_session(runner, "What is the capital of France?", "geography-session")
>>> await run_session(runner, ["Hello!", "What's my name?"], "user-intro-session")
```

In [3]:

async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:

        if type(user_queries) == str:
            user_queries = [user_queries]

       
        for query in user_queries:
            print(f"\nUser > {query}")

          
            query = types.Content(role="user", parts=[types.Part(text=query)])

      
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                
                if event.content and event.content.parts:
                    
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print(" Helper functions defined.")

 Helper functions defined.


### 1.5: Configure Retry Options


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5, 
    exp_base=7,  
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  
)

### 2.4 Implementing Our First Stateful Agent

with a simple Session Management option (`InMemorySessionService`):

In [5]:
APP_NAME = "default"  
USER_ID = "default"  
SESSION = "default"  

MODEL_NAME = "gemini-2.5-flash-lite"



root_agent = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot",  
)

session_service = InMemorySessionService()

runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print(" Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

 Stateful agent initialized!
   - Application: default
   - User: default
   - Using: InMemorySessionService


### 2.5 Testing Our Stateful Agent

Now let's see the magic of sessions in action!

In [6]:

await run_session(
    runner,
    [
        "Hi, I am Sam! What is the capital of United States?",
        "Hello! What is my name?",  
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Hi, I am Sam! What is the capital of United States?
gemini-2.5-flash-lite >  Hi Sam! The capital of the United States is Washington, D.C.

User > Hello! What is my name?
gemini-2.5-flash-lite >  Your name is Sam!


#### Agent's forgetfulness

> To verify that the agent forgets the conversation, **restart the kernel**. Then, **run ALL the previous cells in this notebook EXCEPT the above cell

run the below cell to check the forgetfulness

In [ ]:
# Run this cell after restarting the kernel. All this history will be gone...
await run_session(
    runner,
    ["What did I ask you about earlier?", "And remind me, what's my name?"],
    "stateful-agentic-session",
)  


 ### Session: stateful-agentic-session

User > What did I ask you about earlier?
gemini-2.5-flash-lite >  You asked me about the capital of the United States and what your name is.

User > And remind me, what's my name?
gemini-2.5-flash-lite >  Your name is Sam.


### The Problem 
To achieve this, we must persist information. 

In [7]:
pip install aiosqlite



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Implementing Persistent Sessions

using DatabaseSessionService using SQLite. 



In [ ]:

chatbot_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot with persistent memory",
)

db_url = "sqlite+aiosqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)


runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print(" Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

 Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


#### Verifying Persistence




In [8]:
await run_session(
    runner,
    ["Hi, I am Sam! What is the capital of the United States?", "Hello! What is my name?"],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > Hi, I am Sam! What is the capital of the United States?
gemini-2.5-flash-lite >  Hi Sam! The capital of the United States is Washington, D.C.

User > Hello! What is my name?
gemini-2.5-flash-lite >  Your name is Sam!


#### Resuming a Conversation

> let's stop this Notebook's kernel and restart it again.**
>
> Run all the previous cells in the notebook, EXCEPT the previous Section 3.3 above cell
>
> 2. Now, run the below cell with the **same session ID** (`test-db-session-01`).**Because the session is loaded from the database, the agent should still remember** 


In [9]:
await run_session(
    runner,
    ["What is the capital of India?", "Hello! What is my name?"],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > What is the capital of India?
gemini-2.5-flash-lite >  The capital of India is New Delhi.

User > Hello! What is my name?
gemini-2.5-flash-lite >  Your name is Sam!


### the session data is isolated

a session is private conversation between an Agent and a User. Let's run our `run_session` with a different session name `test-db-session-02` to confirm this.


In [10]:
await run_session(
    runner, ["Hello! What is my name?"], "test-db-session-02"
)  


 ### Session: test-db-session-02

User > Hello! What is my name?
gemini-2.5-flash-lite >  I do not have access to your personal information, including your name. Therefore, I cannot tell you what your name is.


### 3.6 How are the events stored in the Database?

Since we are using a sqlite DB to store information, let's have a quick peek to see how information is stored.

In [11]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

['app_name', 'session_id', 'author', 'content']
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hi, I am Sam! What is the capital of the United States?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Hi Sam! The capital of the United States is Washington, D.C."}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Your name is Sam!"}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "What is the capital of India?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "The capital of India is New Delhi."}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": 

---
#### Section 4: Context Compaction

all the events are stored in full in the session Database, and this quickly adds up. For a long, complex task, this lists events can become very large, leading to slower performance and higher costs.

But what if we could automatically summarize the past? Let's using ADK's **Context Compaction** feature to see 

In [12]:

research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  
        overlap_size=1,  
    ),
)

db_url = "sqlite:///my_agent_data.db"  
session_service = DatabaseSessionService(db_url="sqlite+aiosqlite:///my_agent_data.db")
research_runner_compacting = Runner(
    app=research_app_compacting, session_service=session_service
)


print(" Research App upgraded with Events Compaction!")

 Research App upgraded with Events Compaction!


C:\Users\HP\AppData\Local\Temp\ipykernel_12572\279560604.py:5: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


In [12]:
from google.adk.apps.app import App  
from google.adk.sessions import DatabaseSessionService
from google.adk.runners import Runner

research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,

)


db_url = "sqlite+aiosqlite:///my_agent_data.db"
session_service = DatabaseSessionService(db_url=db_url)


research_runner_compacting = Runner(
    app=research_app_compacting,
    session_service=session_service,
)

print(" Research App using DatabaseSessionService")


 Research App using DatabaseSessionService


In [14]:
SESSION_NAME = "research_db_session_v2"


await run_session(
    research_runner_compacting,
    "What is the latest news about AI in healthcare?",
    SESSION_NAME,
)

await run_session(
    research_runner_compacting,
    "Are there any new developments in drug discovery?",
    SESSION_NAME,
)



 ### Session: research_db_session_v2

User > What is the latest news about AI in healthcare?
gemini-2.5-flash-lite >  Here's a summary of the latest news and trends in AI in healthcare:

**Key Areas of Advancement and Investment:**

*   **Drug Discovery and Development:** This remains a hot area. AI is being used to:
    *   **Identify novel drug targets:** Analyzing vast biological datasets to find new proteins or pathways associated with diseases.
    *   **Design new molecules:** Predicting the properties of potential drug candidates and designing novel compounds with desired efficacy and safety profiles.
    *   **Accelerate clinical trials:** Identifying suitable patient cohorts, predicting trial outcomes, and optimizing trial design, potentially reducing costs and timelines.
    *   **Recent examples:** Companies like Insilico Medicine, BenevolentAI, and Recursion Pharmaceuticals are making significant strides and securing substantial funding.

*   **Medical Imaging and Diagnost

#### Verifying Compaction in the Session History



In [15]:

final_session = await session_service.get_session(
    app_name=research_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="compaction_demo",
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for event in final_session.events:
   
    if event.actions and event.actions.compaction:
        print("\n SUCCESS! Found the Compaction Event:")
        print(f"  Author: {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True
        break

if not found_summary:
    print(
        "\n No compaction event found. Try increasing the number of turns in the demo."
    )

--- Searching for Compaction Summary Event ---

 SUCCESS! Found the Compaction Event:
  Author: user

 Compacted information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='815e1eb7-7801-4ec6-8b80-0261c922f818' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction={'start_timestamp': 1764463265.725412, 'end_timestamp': 1764463273.409589, 'compacted_content': {'parts': [{'media_resolution': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_call': None, 'function_respo